### Installation of cartoon component library

In [ ]:
!git clone https://github.com/tochikuji/Cartoon-Texture-Decomposition
%cd /content/Cartoon-Texture-Decomposition
!pip install .

### Extraction of Cartoon Components

In [ ]:
import glob
import os
import numpy
import cv2
import matplotlib.pyplot as plt

from cartex import CartoonTextureDecomposition
from PIL import Image  
import PIL
# Print png images in folder C:\Users\admin\
folder_name = "more_texture/"
Source_folder_name = "/content/drive/MyDrive/LatentImages/more_textures/"
dest_folder_name = "/content/drive/MyDrive/LatentImages/cartoon-component/"+folder_name
if not os.path.exists(dest_folder_name):
    os.makedirs(dest_folder_name)
for filepath in glob.iglob(r''+Source_folder_name+'*.*'):
    print(filepath)
    filename = filepath#'/content/Cartoon-Texture-Decomposition/examples/docimg.jpg'
    # Grayscale images
    img = cv2.imread(filename, 0)
    # create decomposer object
    obj = CartoonTextureDecomposition(sigma=2, ksize=7, n_iter=5)
    # decompose into cartoon and texture components
    cartoon, texture = obj.decompose(img)
    # colored ones
    img_color = cv2.imread(filename)
    cartoon_color, texture_color = obj.decompose(img_color)
    mlp = lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    # save a image using extension 
    plt.imsave(dest_folder_name+os.path.basename(filepath), cartoon, cmap='gray')





### Generation of Synthetic data

In [ ]:
import glob
import os
import numpy
import cv2
import matplotlib.pyplot as plt
import re
from cartex import CartoonTextureDecomposition 
import PIL
from PIL import Image, ImageOps
import random
import gc

# Print png images in folder C:\Users\admin\
#folder_name = "more_texture/"
cartoon_folder_name = "/content/drive/MyDrive/LatentImages/cartoon_components_final/"
fp_folder_name = "/content/drive/MyDrive/LatentImages/roll/png/"
dest_synthetic_folder_name = "/content/drive/MyDrive/LatentImages/CompleteData/synthetic/"
dest_fp_mask_folder_name = "/content/drive/MyDrive/LatentImages/CompleteData/mask/"
if not os.path.exists(dest_synthetic_folder_name):
    os.makedirs(dest_synthetic_folder_name)
if not os.path.exists(dest_fp_mask_folder_name):
    os.makedirs(dest_fp_mask_folder_name)
folder = 0
for filepath in glob.iglob(r''+fp_folder_name+'*.*'):
  #print(filepath)
  filename = os.path.basename(filepath)
  filename = re.sub(r'\.jpg$', '', filename)
  for filepath_cartoon in glob.iglob(r''+cartoon_folder_name+'*.*'):
    #print(filepath_cartoon)
    if not os.path.exists("/content/drive/MyDrive/LatentImages/CompleteData/synthetic/"+filename):
      os.makedirs("/content/drive/MyDrive/LatentImages/CompleteData/synthetic/"+filename)
    if not os.path.exists("/content/drive/MyDrive/LatentImages/CompleteData/mask/"+filename):
      os.makedirs("/content/drive/MyDrive/LatentImages/CompleteData/mask/"+filename)
      
    #reading of iamges
    folder=folder+1
    cartoon_img = Image.open(filepath_cartoon)
    #print(cartoon_img.mode,cartoon_img.size,cartoon_img.format)
    plt.imshow(cartoon_img,cmap='gray')
    fp_img = Image.open(filepath).convert(cartoon_img.mode)
    # padding of fp images
    fp_img_with_border = ImageOps.expand(fp_img,border=50,fill='white')
    fp_img_with_border = fp_img_with_border.resize(cartoon_img.size)
    #print(fp_img_with_border.mode,fp_img_with_border.size,fp_img_with_border.format)
    fp_img = fp_img_with_border#Image.open("/content/Cartoon-Texture-Decomposition/examples/00002223_01_dryrun-A_500_roll_01.png").convert(Im.mode)
    #print(fp_img.mode,fp_img.size,fp_img.format)
    fp_img = fp_img.resize(cartoon_img.size)
    #print(fp_img.mode,fp_img.size,fp_img.format)


#######################
    #bleding of image
    w = round(random.uniform(0.3, 0.7), 1)
    img = Image.blend(cartoon_img,fp_img,w)
##################
# thresholding of fp
    fp_img = cv2.cvtColor(numpy.array(fp_img), cv2.COLOR_RGB2GRAY)
    thresholded_fp = cv2.adaptiveThreshold(fp_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    plt.imshow(thresholded_fp,cmap='gray')
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(12,12))
    thresholded_fp = cv2.morphologyEx(thresholded_fp, cv2.MORPH_OPEN, kernel)
    thresholded_fp = cv2.bitwise_not(thresholded_fp)
    fp_thresh_im_pil = Image.fromarray(thresholded_fp)
###################

#saving of Image
    img_to_save = img.resize((816,816))
    img_to_save_fp = fp_thresh_im_pil.resize((816,816))
    
    img_to_save.save("/content/drive/MyDrive/LatentImages/CompleteData/synthetic/"+filename+"/synthetic_img"+str(folder)+".png")
    img_to_save_fp.save("/content/drive/MyDrive/LatentImages/CompleteData/mask/"+filename+"/fp_mask_img"+str(folder)+".png")
################
    print(folder)
  biglist = [img_to_save,img_to_save_fp,fp_img,thresholded_fp,cartoon_img]
  gc.collect()
  del biglist
    





Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002264_4E_X_106_IN_D800_1112PPI_16BPC_1CH_LP07_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002264_1A_L_L01_BP_S03_1200PPI_8BPC_1CH_LP01_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002264_1H_X_L01_BP_S03_1200PPI_8BPC_1CH_LP05_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002264_1C_R_L01_BP_S03_1200PPI_8BPC_1CH_LP01_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002264_1C_R_L01_BP_S03_1200PPI_8BPC_1CH_LP03_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002267_1A_R_L01_BP_S03_1200PPI_8BPC_1CH_LP03_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002267_1A_R_L01_BP_S03_1200PPI_16BPC_1CH_LP03_1.png
/content/drive/MyDrive/LatentImages/cartoon_components_final/00002267_1A_L_L01_BP_S03_1200PPI_8BPC_1CH_LP03_1.png
/content/drive/MyDrive/LatentImage